In [404]:
import numpy as np
from shapely.geometry import LineString, Polygon
import geopandas as gpd

In [405]:
# Read in linestring object
file = '/Users/hyin/usgs_mendenhall/ffsimmer/fault-extrusion/2025_kamchatka/kamchatka-trace.shp'

# extract file base name (without extension) for later use
filename = file.split('/')[-1].split('.')[0]
print(f"Processing fault file: {filename}")
dir = '/'.join(file.split('/')[:-1])
print(f"Directory: {dir}")

zmax = 40.0 # maximum depth for extrusion (km)
dip_deg = 18 # dip angle in degrees

trace = gpd.read_file(file)

# Convert to local UTM
utm_crs = trace.to_crs(trace.estimate_utm_crs()).crs
trace = trace.to_crs(utm_crs)

## Extract the individual segments from each linestring
line = trace.geometry.values[0] # assumes there's only one linestring in the shapefile
coords = list(line.coords)
print(coords)

# ## Convert to local UTM coordinates for extrusion
# utm_crs = trace.to_crs(trace.estimate_utm_crs()).crs
# print(f"Using local UTM CRS: {utm_crs}")
# trace_utm = trace.to_crs(utm_crs)
# line_utm = trace_utm.geometry.values[0]
# coords_utm = list(line_utm.coords)

# print(coords_utm)
# line_utm = trace_utm.geometry.values[0]




Processing fault file: kamchatka-trace
Directory: /Users/hyin/usgs_mendenhall/ffsimmer/fault-extrusion/2025_kamchatka
[(669173.1423434948, 5804284.776345385), (369969.15414039115, 5420202.983495098)]


In [406]:
def strike_and_dipdir(p0, p1):
    '''
    Calculate strike and dip direction from two points using the right-hand rule.
    '''
    dx = p1[0] - p0[0]
    dy = p1[1] - p0[1]
    strike = np.degrees(np.arctan2(dx, dy)) % 360
    dipdir = (strike + 90) % 360
    return strike, dipdir


In [407]:
segment_data = []   # Initialize a list of segments with their extrusion vectors

for i in range(len(coords) - 1):
    start = coords[i]
    end = coords[i + 1]

    print(f"Segment {i}: start={start}, end={end}")

    strike, dipdir = strike_and_dipdir(start, end)
    print(f"Strike: {strike:.2f} degrees")
    print(f"Dip Direction: {dipdir:.2f} degrees")

    # Convert the dip direction to a unit vector in the horizontal plane (east, north)
    dip_rad = np.radians(dipdir)
    dip_xy = np.array([np.sin(dip_rad), np.cos(dip_rad)]) # east, north
    dip_xy /= np.linalg.norm(dip_xy)    # normalize to unit vector to get dip direction unit vector
    print(f"Dip direction unit vector (dip_xy): {dip_xy}")
    L_h = zmax / np.tan(np.radians(dip_deg))
    print(f"Horizontal extrusion length (L_h): {L_h:.2f} km")
    
    V = np.array([
        dip_xy[0] * L_h*1000,
        dip_xy[1] * L_h*1000,
        -zmax*1000
    ])
    print(f"Extrusion vector (V): {V}\n")

    segment_data.append({
    "start": start,
    "end": end,
    "V": V
    })

# example of how to access the data for the first segment
segment_data[0]['start']        # each segment has 'start', 'end', and 'V' keys

Segment 0: start=(669173.1423434948, 5804284.776345385), end=(369969.15414039115, 5420202.983495098)
Strike: 217.92 degrees
Dip Direction: 307.92 degrees
Dip direction unit vector (dip_xy): [-0.78888047  0.61454666]
Horizontal extrusion length (L_h): 123.11 km
Extrusion vector (V): [-97116.97775896  75655.20576249 -40000.        ]



(669173.1423434948, 5804284.776345385)

## Deal with the extrusion conflicts at depth
For each "vertex" (intersection between two line segments) we need to find a common point at depth where the two planes can intersect

In [408]:
coords[0]

(669173.1423434948, 5804284.776345385)

In [409]:
P_bottom = {}
P_top = {}

# Calculate the first vertex
P0 = np.array([coords[0][0], coords[0][1], 0.0])    # Make surfaace point 3D by adding z=0

P_bottom[0] = P0 + segment_data[0]["V"]
P_top[0] = P0


# Calculate the middle vertices
for i in range(1, len(coords) - 1):
    Pi = np.array([*coords[i], 0.0])

    V_left  = segment_data[i - 1]["V"]
    V_right = segment_data[i]["V"]

    P_left  = Pi + V_left
    P_right = Pi + V_right

    P_bottom[i] = 0.5 * (P_left + P_right)
    P_top[i]    = Pi

# Calculate the last vertex
Pn = np.array([coords[-1][0], coords[-1][1], 0.0])
P_bottom[len(coords)-1] = Pn + segment_data[-1]["V"]
P_top[len(coords)-1] = Pn


from pyproj import Transformer

wgs84_epsg = "EPSG:4326"

transformer = Transformer.from_crs(
    utm_crs,
    wgs84_epsg,
    always_xy=True   # VERY IMPORTANT: (x, y) = (lon, lat)
)

def transform_points_dict(points_dict, transformer):
    """
    Convert {index: np.array([x, y, z])} from UTM to WGS84.
    Returns {index: (lon, lat, z)} with Python floats.
    """
    out = {}

    for i, p in points_dict.items():
        x, y, z = p
        lon, lat = transformer.transform(x, y)
        out[i] = (float(lon), float(lat), float(z))

    return out


P_top_ll = transform_points_dict(P_top, transformer)
P_bottom_ll = transform_points_dict(P_bottom, transformer)
print(P_bottom_ll)
print(P_top_ll)


{0: (160.07530672121786, 53.06408498715374, -40000.0), 1: (155.85802647353714, 49.57262495224662, -40000.0)}
{0: (161.4845906560444, 52.3626937261775, 0.0), 1: (157.22499011648654, 48.92109610053944, 0.0)}


# Build subfault polygons

In [410]:
polys = []

for i in range(len(coords) - 1):
    P0 = np.array(coords[i])
    P1 = np.array(coords[i + 1])

    B0 = P_bottom[i]
    B1 = P_bottom[i + 1]

    poly = Polygon([
        (P0[0], P0[1], 0.0),
        (P1[0], P1[1], 0.0),
        (B1[0], B1[1], B1[2]),
        (B0[0], B0[1], B0[2]),
        (P0[0], P0[1], 0.0)
    ])

    polys.append(poly)

polys

[<POLYGON Z ((669173.142 5804284.776 0, 369969.154 5420202.983 0, 272852.176 ...>]

In [411]:
gdf_out = gpd.GeoDataFrame(
    geometry=polys,
    crs=utm_crs).to_crs("EPSG:4326")   # Convert back to WGS84 for output

gdf_out.to_file(f"{dir}/{filename}_dip{dip_deg}_z{str(zmax)}km_extruded.geojson", driver="GeoJSON")


## Geojson to `fault.json`
Write fault.json file in the format ShakeMap expects.
- A single MultiPolygon 
- Coordinates in the order surface trace, trace at depth, back to starting point. 

```
P0 ── P1 ── P2 ── ... ── PN
│                         │
│                         │
PN' ── ... ── P2' ── P1' ── P0'
```

In [412]:
def build_fault_polygon(surface_trace, bottom_trace):
    """
    Returns a single polygon coordinate array suitable for GeoJSON MultiPolygon.
    """

    if len(surface_trace) != len(bottom_trace):
        raise ValueError("Surface and bottom traces must have same length")

    # 1. Surface trace (forward)
    coords = list(surface_trace)

    # 2. Bottom trace (reverse)
    coords.extend(reversed(bottom_trace))

    # 3. Close polygon
    coords.append(surface_trace[0])

    return coords

In [413]:
surface_trace = [
    tuple(map(float, P_top_ll[i]))
    for i in sorted(P_top_ll)
]

depth_trace = [
    tuple(map(float, P_bottom_ll[i]))
    for i in sorted(P_bottom_ll)
]

print(depth_trace[0])

(160.07530672121786, 53.06408498715374, -40000.0)


In [414]:
coords_fault = build_fault_polygon(surface_trace, depth_trace)

In [415]:
coords_fault

[(161.4845906560444, 52.3626937261775, 0.0),
 (157.22499011648654, 48.92109610053944, 0.0),
 (155.85802647353714, 49.57262495224662, -40000.0),
 (160.07530672121786, 53.06408498715374, -40000.0),
 (161.4845906560444, 52.3626937261775, 0.0)]

## Write to JSON   

In [416]:
# metadata = f"Test"
outfile = f"{dir}/{filename}_dip{dip_deg}_z{str(zmax)}km_shakemap.json"


In [417]:
polygon_coords = build_fault_polygon(surface_trace, depth_trace)

import json

reference=" "
id=" "
network=""
netid=" "
productcode=" "
time=" "
lat=0.0
lon=0.0
depth=0.0
mag=0.0
locstring=" "
mech=" "
reviewed=" "
rake=0.0
# def write_fault_geojson(
#     surface_trace,
#     bottom_trace,
#     outfile,
#     metadata,
# ):

geojson = {
    "type": "FeatureCollection",
    "metadata":  {
        "reference": reference,
        "id": id,
        "network": network,
        "netid": netid,
        "productcode": productcode,
        "time": time,
        "lat": lat,
        "lon": lon,
        "depth": depth,
        "mag": mag,
        "locstring": locstring,
        "mech": mech,
        "reviewed": reviewed,
        "rake": rake
    },
    "features": [
        {
            "type": "Feature",
            "properties": {
                "rupture type": "rupture extent"
            },
            "geometry": {
                "type": "MultiPolygon",
                "coordinates": [
                    [
                        polygon_coords
                    ]
                ]
            }
        }
    ]
}

with open(outfile, "w") as f:
    json.dump(geojson, f, indent=4)